In [3]:
import cv2
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
import tensorflow as tf
from tqdm import tqdm

In [4]:
def build_model(pretrained):
    model = Sequential([
        pretrained,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(),
        metrics=['accuracy']
    )
    
    return model

# Generator

In [7]:
base_path = 'C:\\Users\\Deep\\Downloads\\Images\\archive\\real_vs_fake\\real-vs-fake'

image_gen = ImageDataGenerator(rescale=1./255.,
                               rotation_range=20,
                               #shear_range=0.2,
                               #zoom_range=0.2,
                               horizontal_flip=True)

train_flow = image_gen.flow_from_directory(
    base_path + '\\train\\',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)




Found 100000 images belonging to 2 classes.


In [8]:
image_gen1 = ImageDataGenerator(rescale=1./255.)

valid_flow = image_gen1.flow_from_directory(
    base_path + '\\valid\\',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


Found 20000 images belonging to 2 classes.


# Train Model DenseNet Augmented Data 

In [9]:
densenet = DenseNet121(
    weights=None,
    include_top=False,
    input_shape=(224,224,3)
)
model = build_model(densenet)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 1025      
                                                                 
Total params: 7,038,529
Trainable params: 6,954,881
Non-trainable params: 83,648
_________________________________________________________________


In [11]:
train_steps = 100000//64
valid_steps = 20000//64

history = model.fit_generator(
    train_flow,
    epochs = 20,
    steps_per_epoch= train_steps,
    validation_data= valid_flow,
    validation_steps= valid_steps
)

model.save('.\\models\\DenseNet_Augmented_model.h5')

C:\Users\Deep\AppData\Local\Temp\ipykernel_22636\3677287013.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/20
1562/1562 [==============================] - 416s 263ms/step - loss: 0.6774 - accuracy: 0.5787 - val_loss: 1.2765 - val_accuracy: 0.4980
Epoch 2/20
1562/1562 [==============================] - 415s 266ms/step - loss: 0.5865 - accuracy: 0.6873 - val_loss: 0.5993 - val_accuracy: 0.6956
Epoch 3/20
1562/1562 [==============================] - 416s 266ms/step - loss: 0.4889 - accuracy: 0.7660 - val_loss: 0.5081 - val_accuracy: 0.7632
Epoch 4/20
1562/1562 [==============================] - 453s 290ms/step - loss: 0.4216 - accuracy: 0.8091 - val_loss: 0.4232 - val_accuracy: 0.8021
Epoch 5/20
1562/1562 [==============================] - 434s 278ms/step - loss: 0.3758 - accuracy: 0.8346 - val_loss: 0.5210 - val_accuracy: 0.7652
Epoch 6/20
1562/1562 [==============================] - 430s 275ms/step - loss: 0.3323 - accuracy: 0.8566 - val_loss: 0.3347 - val_accuracy: 0.8578
Epoch 7/20
1562/1562 [==============================] - 416s 267ms/step - loss: 0.3004 - accuracy: 0.8727 - val_

# Evaluation

In [12]:
test_flow = image_gen1.flow_from_directory(
    base_path + '\\test\\',
    target_size=(224, 224),
    batch_size=1,
    shuffle = False,
    class_mode='binary'
)
y_pred=model.predict(test_flow)
y_test = test_flow.classes

Found 20000 images belonging to 2 classes.
20000/20000 [==============================] - 352s 18ms/step


In [13]:
print("ROC AUC Score:", metrics.roc_auc_score(y_test, y_pred))
print("AP Score:", metrics.average_precision_score(y_test, y_pred))
print()
print(metrics.classification_report(y_test, y_pred > 0.5))

ROC AUC Score: 0.968640695
AP Score: 0.966609199565477

              precision    recall  f1-score   support

           0       0.96      0.81      0.88     10000
           1       0.83      0.97      0.90     10000

    accuracy                           0.89     20000
   macro avg       0.90      0.89      0.89     20000
weighted avg       0.90      0.89      0.89     20000

